# Difference analysis between AI generated en final letter

Analysis to see how many changes were made to the AI-draft discharge letter in comparison to the final discharge letter sent to the next treating physician.

### Import dependencies

In [ ]:
import os
import re
from datetime import date
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
from nltk.util import ngrams
from rich import print as rprint
from tqdm.notebook import tqdm

tqdm.pandas()
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWD = os.getenv("DB_PASSWORD")


### Export data to datamanager folder
- Export database tables (request, requestdischarge, request..., encounter, generateddoc) to csv files. 
- Export data_export.parquet by running data_pipeline.py (export & processing = True, bulk generate = False).
        Change data.to_parquet(Path(processed_data_folder / "evaluation_data.parquet")) to new datamanager folder. 
- Export final_discharge_letters.csv by running metavision_discharge_docs_retro.sql (if only metavision is used). Change the period start and end. 

### Load data

In [ ]:
# load data from application database export file
data_folder = Path(
    "/mapr/administratielast/administratielast_datamanager/ontslagdocumentatie/PMS_1_IC_NICU"
)

date_analysis = "2025-07-15"
request_retrieve = pd.read_csv(
    data_folder / Path(date_analysis + "-requestretrieve.csv")
)
request_generate = pd.read_csv(
    data_folder / Path(date_analysis + "-requestgenerate.csv")
)
request = pd.read_csv(
    data_folder / Path(date_analysis + "-request.csv"),
    engine="pyarrow",
    parse_dates=["timestamp"],
)
encounter = pd.read_csv(data_folder / Path(date_analysis + "-encounter.csv"))
generateddoc = pd.read_csv(data_folder / Path(date_analysis + "-generateddoc.csv"))
data = pd.read_parquet(data_folder / Path("data_export_pms_1.parquet"))
final_discharge = pd.read_csv(data_folder / "final_discharge_letters.csv")

display(request_retrieve.head())
display(request_generate.head())
display(request.head())
display(encounter.head())
display(generateddoc.head())
display(data.head())
display(final_discharge.head())

In [ ]:
START_DATE = date(2025, 4, 1)
END_DATE = date(2025, 7, 1)
DEPARTMENTS = ["IC", "NICU"]

### Create generated_doc_merged
This table is the result of a merge between the encounter, generateddoc, request_generate, and request tables. It holds all information on the generated documents combined with the generated requests. 

In [ ]:
generated_doc_merged = (
    pd.merge(
        encounter,
        generateddoc,
        left_on="id",
        right_on="encounter_id",
        how="inner",
        suffixes=("_encounter", "_generateddoc"),
    )
    .drop(columns="encounter_id")
    .rename(columns={"id_generateddoc": "generated_doc_id"})
)


generated_doc_merged = generated_doc_merged.join(
    request_generate.set_index("id"),
    on="request_generate_id",
    rsuffix="_request_generate",
)

generated_doc_merged = generated_doc_merged.join(
    request.set_index("id"),
    on="request_id",
    rsuffix="_request",
)

generated_doc_merged["timestamp"] = pd.to_datetime(generated_doc_merged["timestamp"])

generated_doc_merged = generated_doc_merged[
    (generated_doc_merged["timestamp"].dt.date >= START_DATE)
    & (generated_doc_merged["timestamp"].dt.date <= END_DATE)
]

generated_doc_merged = generated_doc_merged[
    generated_doc_merged["department"].isin(DEPARTMENTS)
    | generated_doc_merged["department"].isnull()
]

display(generated_doc_merged.head())

### Create request_retrieve_merged
This table is the result of a merge between encounter, request_retrieve, and request. 

In [ ]:
request_retrieve_merged = pd.merge(
    request_retrieve,
    request,
    left_on="request_id",
    right_on="id",
    how="inner",
    suffixes=("_request", "_encounter"),
)

request_retrieve_merged = request_retrieve_merged.join(
    encounter.set_index("enc_id"),
    on="request_enc_id",
    rsuffix="_encounter",
    how="outer",
)

request_retrieve_merged["timestamp"] = pd.to_datetime(
    request_retrieve_merged["timestamp"]
)

request_retrieve_merged = request_retrieve_merged[
    (request_retrieve_merged["timestamp"].dt.date >= START_DATE)
    & (request_retrieve_merged["timestamp"].dt.date <= END_DATE)
]

request_retrieve_merged = request_retrieve_merged[
    request_retrieve_merged["department"].isin(DEPARTMENTS)
    | request_retrieve_merged["department"].isnull()
]

display(request_retrieve_merged.head())

### Generated doc outcomes

In [ ]:
number_of_docs_generated = generated_doc_merged["generated_doc_id"].nunique()
rprint(f"Number of generated docs: {number_of_docs_generated}")

number_of_successful_docs = generated_doc_merged.loc[
    generated_doc_merged["success_ind"] == "Success"
]["generated_doc_id"].nunique()
rprint(f"Number of successfully generated docs: {number_of_successful_docs}")

number_of_encounters = generated_doc_merged["enc_id"].nunique()
rprint(f"Number of unique encounters: {number_of_encounters}")


failure_reasons = generated_doc_merged[
    generated_doc_merged["success_ind"] != "Success"
]["success_ind"].value_counts()
rprint(f"Reasons for not successful generation: {failure_reasons}")

perc_generateddoc_too_long = failure_reasons.LengthError / number_of_docs_generated
rprint(
    "Percentage of letters that were not generated because file was too long: "
    f"{perc_generateddoc_too_long * 100:.2f} %"
)

length_error_encounters = generated_doc_merged[
    generated_doc_merged["success_ind"] == "LengthError"
]["enc_id"].nunique()
rprint(f"Number of unique encounters with length error: {length_error_encounters}")

perc_length_error_encounters = (length_error_encounters / number_of_encounters) * 100
rprint(
    "Percentage of unique encounters with length error: "
    f"{perc_length_error_encounters:.2f} %"
)


### Request outcomes

In [ ]:
number_of_processed_requests = generated_doc_merged["request_id"].count()
rprint(f"Number of processed requests: {number_of_processed_requests}")

number_of_unique_processed_requests = generated_doc_merged["request_id"].nunique()
rprint(f"Number of unique processed requests: {number_of_unique_processed_requests}")

number_of_retrieved_requests = request_retrieve_merged["request_id"].nunique()
rprint(f"Number of retrieved requests: {number_of_retrieved_requests}")

number_of_successfully_retrieved_requests = request_retrieve_merged[
    request_retrieve_merged["success_ind"] != 0
]["request_id"].nunique()
rprint(
    f"Number of successfully retrieved requests: "
    f"{number_of_successfully_retrieved_requests}"
)

percentage_successfully_retrieved_requests = (
    number_of_successfully_retrieved_requests / number_of_retrieved_requests * 100
)
rprint(
    "Percentage of successfully retrieved requests: "
    f"{percentage_successfully_retrieved_requests:.2f} %"
)

number_of_encounters_with_successfully_retrieved_requests = request_retrieve_merged[
    request_retrieve_merged["success_ind"] != 0
]["request_enc_id"].nunique()
rprint(
    "Number of unique encounters with successfully retrieved requests: "
    f"{number_of_encounters_with_successfully_retrieved_requests}"
)

percentage_encounters_with_successfully_retrieved_requests = (
    number_of_encounters_with_successfully_retrieved_requests
    / number_of_encounters
    * 100
)
rprint(
    "Percentage of unique encounters with successfully retrieved requests: "
    f"{percentage_encounters_with_successfully_retrieved_requests:.2f} %"
)

### Combine generated docs with Metavision docs

In [ ]:
metavision_discharge_docs = (
    data.loc[data["description"] == "Ontslagbrief"]
    .sort_values("date", ascending=True)
    .drop_duplicates(subset="enc_id", keep="last")
)
# Metavision letters use \r\n for newlines or double newlines and sometimes add ...
# at the end of a sentence
metavision_discharge_docs["content"] = metavision_discharge_docs["content"].str.replace(
    "\r", ""
)
metavision_discharge_docs["content"] = metavision_discharge_docs["content"].str.replace(
    "\n\n\n", "\n\n"
)
metavision_discharge_docs["content"] = metavision_discharge_docs["content"].str.replace(
    "...", ""
)

display(metavision_discharge_docs.head())

generated_doc_merged = generated_doc_merged.sort_values(
    "timestamp", ascending=True
).drop_duplicates(subset="enc_id", keep="last")

merged_discharge_docs = generated_doc_merged.join(
    metavision_discharge_docs.set_index("enc_id"),
    on="enc_id",
    rsuffix="_metavision",
)

display(merged_discharge_docs.head())


### Filtering

In [ ]:
# print number of unique encounters in merged discharge docs
number_of_unique_encounters_in_merged_docs = merged_discharge_docs["enc_id"].nunique()
rprint(
    f"Number of unique encounters in merged discharge docs: "
    f"{number_of_unique_encounters_in_merged_docs}"
)

#  Filter out rows where the document was not successfully generated
merged_discharge_docs = merged_discharge_docs[
    merged_discharge_docs["success_ind"] == "Success"
]

number_of_unique_encounters_in_merged_docs = merged_discharge_docs["enc_id"].nunique()
rprint(
    f"Number of unique encounters with successfully generated discharge docs: "
    f"{number_of_unique_encounters_in_merged_docs}"
)

# Filter out letters where the generated document was never requested
request_retrieve_merged_successfully = request_retrieve_merged[
    request_retrieve_merged["success_ind"] != 0
]

enc_ids_successfully_retrieved = request_retrieve_merged_successfully[
    "request_enc_id"
].unique()
rprint(
    f"Number of unique encounters with successfully retrieved requests: "
    f"{len(enc_ids_successfully_retrieved)}"
)

merged_discharge_docs = merged_discharge_docs[
    merged_discharge_docs["enc_id"].isin(enc_ids_successfully_retrieved)
]

number_of_unique_encounters_in_merged_docs = merged_discharge_docs["enc_id"].nunique()
rprint(
    "Number of unique encounters with successfully retrieved requests in merged "
    f"discharge docs: {number_of_unique_encounters_in_merged_docs}"
)

# Filter out discharge docs with a missing description
merged_discharge_docs = merged_discharge_docs[
    merged_discharge_docs["description"].notna()
]

rprint(
    "Number of unique encounters with successfully generated discharge docs and "
    f"a original letter in metavision: {merged_discharge_docs['enc_id'].nunique()}"
)

display(merged_discharge_docs.head())

### Check differences between generated and Metavision docs

In [ ]:
def longest_common_substring(generated_letter: str, original_letter: str) -> str:
    """Finds the longest common substring between two strings
    using dynamic programming."""
    generated_letter = generated_letter.lower()
    original_letter = original_letter.lower()
    m, n = len(generated_letter), len(original_letter)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    max_length = 0
    end_index = 0  # End index of the longest substring in generated_letter

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if generated_letter[i - 1] == original_letter[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end_index = i

    return generated_letter[end_index - max_length : end_index]


longest_common_substring("dit is een \ntest zin", "is maar een \nTester")

In [ ]:
def highlight_lcs(
    df: pd.DataFrame,
    enc_id: int,
    content_column: str = "content",
    discharge_letter_column: str = "discharge_letter",
) -> None:
    """retrieves the row for the given encounter id and
    highlights the longest common substring using rich"""
    enc_row = df.loc[df["enc_id"] == enc_id]

    original_letter = enc_row[content_column].to_numpy()[0]
    generated_letter = enc_row[discharge_letter_column].to_numpy()[0]

    lcs = longest_common_substring(generated_letter, original_letter)

    # Use rich to highlight the longest common substring in both letters
    replace_pattern = re.compile(re.escape(lcs), re.IGNORECASE)
    original_letter = replace_pattern.sub(
        f"[italic green]{lcs}[/italic green]", original_letter
    )
    generated_letter = replace_pattern.sub(
        f"[italic green]{lcs}[/italic green]", generated_letter
    )
    rprint("[bold yellow]Original letter[/bold yellow]")
    rprint(original_letter)
    rprint("[bold yellow]Generated letter[/bold yellow]")
    rprint(generated_letter)


highlight_lcs(merged_discharge_docs, 10705)


In [ ]:
def lcs_distance(generated_letter: str, original_letter: str) -> float:
    """Calculate the longest common substring distance between two strings

    Score of 0 means identical texts, while a score of 1 means no common substrings.
    """
    lcs = longest_common_substring(generated_letter, original_letter)
    longest_text = max(len(generated_letter), len(original_letter))
    return (longest_text - len(lcs)) / longest_text


merged_discharge_docs["lcs_distance"] = merged_discharge_docs.progress_apply(
    lambda x: lcs_distance(x["discharge_letter"], x["content"]), axis=1
)  # type: ignore

In [ ]:
def jaccard_distance(generated_letter: str, original_letter: str, n: int) -> float:
    """Calculate the Jaccard distance between two strings using n-grams

    Score of 0 means identical texts, while a score of 1 means no common n-grams.
    """
    generated_letter_words = generated_letter.lower().split()
    original_letter_words = original_letter.lower().split()

    ngrams_generated = set(ngrams(generated_letter_words, n))
    ngrams_original = set(ngrams(original_letter_words, n))

    ngrams_union = ngrams_generated.union(ngrams_original)
    if len(ngrams_union) == 0:
        return 0
    ngrams_intersection = ngrams_generated.intersection(ngrams_original)
    return 1 - len(ngrams_intersection) / len(ngrams_union)


merged_discharge_docs["ngram_1"] = merged_discharge_docs.apply(
    lambda x: jaccard_distance(x["discharge_letter"], x["content"], 1), axis=1
)
merged_discharge_docs["ngram_2"] = merged_discharge_docs.apply(
    lambda x: jaccard_distance(x["discharge_letter"], x["content"], 2), axis=1
)
merged_discharge_docs["ngram_3"] = merged_discharge_docs.apply(
    lambda x: jaccard_distance(x["discharge_letter"], x["content"], 3), axis=1
)
merged_discharge_docs


### Display best matching rows

In [ ]:
merged_discharge_docs.sort_values("ngram_3", ascending=True).head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
merged_discharge_docs["ngram_3"].plot.hist(ax=ax, bins=20)
ax.set_title("Jaccard distance between generated and original letters")
ax.set_xlabel("Jaccard distance")
fig.show()

In [ ]:
merged_discharge_docs.sort_values("lcs_distance", ascending=True).head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
merged_discharge_docs["lcs_distance"].plot.hist(ax=ax, bins=20)
ax.set_title("Longest common substring distance between generated and original letters")
ax.set_xlabel("LCS distance")
fig.show()

In [ ]:
ngram_3_groups = pd.cut(
    merged_discharge_docs["ngram_3"],
    bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1],
)
ngram_3_groups.value_counts().sort_index()


In [ ]:
lcs_groups = pd.cut(
    merged_discharge_docs["lcs_distance"],
    bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
)
lcs_groups.value_counts().sort_index()

### Inspect best matching letters

In [ ]:
def find_closest_example(
    df: pd.DataFrame,
    threshold_score: float,
    score_col: str = "ngram_3",
    col1: str = "content",
    col2: str = "discharge_letter",
) -> None:
    """Find Closes example to the given threshold score

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe containing the discharge letters
    ngram_score : float
        The ngram score to use as a threshold
    """
    sorted_df = df.loc[
        df[score_col] > threshold_score, ["enc_id", score_col]
    ].sort_values(score_col, ascending=True)
    enc_id = sorted_df["enc_id"].iat[0]
    actual_score = sorted_df[score_col].iat[0]
    rprint(f"[bold]Encounter ID: {enc_id}, {score_col} score: {actual_score}[/bold]")
    highlight_lcs(df, enc_id, col1, col2)

In [ ]:
find_closest_example(merged_discharge_docs, 0)

In [ ]:
find_closest_example(merged_discharge_docs, 0.4)

In [ ]:
find_closest_example(merged_discharge_docs, 0.5)

In [ ]:
find_closest_example(merged_discharge_docs, 0.6)

In [ ]:
find_closest_example(merged_discharge_docs, 0.7)

In [ ]:
find_closest_example(merged_discharge_docs, 0.8)

In [ ]:
find_closest_example(merged_discharge_docs, 0.9)

In [ ]:
find_closest_example(merged_discharge_docs, 0.95)

### Check differences Metavision letter and final discharge letter

In [ ]:
final_discharge_filtered = (
    final_discharge.sort_values("date", ascending=False)
    .drop_duplicates(subset="enc_id", keep="first")
    .rename(columns={"content": "final_doc"})[["enc_id", "final_doc"]]
)
final_merged_docs = merged_discharge_docs.merge(
    final_discharge_filtered, on="enc_id", how="left"
)
# Somehow some of the final letters contain floats and Nans..
final_merged_docs = final_merged_docs.dropna(subset=["final_doc"])
final_merged_docs["final_doc"] = final_merged_docs["final_doc"].astype(str)
final_merged_docs

In [ ]:
def ngram_dist_final(metavision_letter: str, final_letter: str, n: int) -> float:
    """Updated ngram distance that only compares the intersection
    with the metavision letters"""
    metavision_letter_words = metavision_letter.lower().split()
    final_letter_words = final_letter.lower().split()

    ngrams_metavision = set(ngrams(metavision_letter_words, n))
    ngrams_final = set(ngrams(final_letter_words, n))

    ngrams_intersection = ngrams_metavision.intersection(ngrams_final)
    if len(ngrams_metavision) == 0:
        return 1
    return 1 - len(ngrams_intersection) / len(ngrams_metavision)


final_merged_docs["ngram_3_final"] = final_merged_docs.progress_apply(
    lambda x: ngram_dist_final(x["content"], x["final_doc"], 3), axis=1
)  # type: ignore

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
final_merged_docs["ngram_3_final"].plot.hist(ax=ax, bins=20)
ax.set_title("Percentage verschil in 3-grams tussen metavision en laatste brief")
ax.set_xlabel("Percentage verschil")
fig.show()

In [ ]:
final_ngram_bins = pd.cut(
    final_merged_docs["ngram_3_final"],
    bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
)
final_ngram_bins.value_counts().sort_index()

In [ ]:
find_closest_example(final_merged_docs, 0.4, "ngram_3_final", "content", "final_doc")